# Vehicle Data Scraping

Source: https://pythoninoffice.com/get-table-data-from-web-page-using-python-pandas/
        https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.notna.html#pandas.DataFrame.notna  
        https://www.fueleconomy.gov/feg/PowerSearch

In [1]:
import pandas as pd
import numpy as np
import re

fueleconomy.gov has tabulated data for all kinds of automobiles. Let's use the `read_html` function in `pandas` to scrape the website for this data. Notice these URLs are quite long. This is because I have several filters from the powersearch feature on the website activated to exclude luxury vehicle brands and focus on vehicle types that are in the UIUC fleet.

In [2]:
diesel = pd.read_html('https://www.fueleconomy.gov/feg/PowerSearch.do?action=PowerSearch&year1=2018&year2=2021&cbmkAcura=Acura&cbmkBYD=BYD&cbmkChevrolet=Chevrolet&cbmkChrysler=Chrysler&cbmkDodge=Dodge&cbmkFiat=Fiat&cbmkFord=Ford&cbmkGMC=GMC&cbmkHonda=Honda&cbmkHyundai=Hyundai&cbmkJeep=Jeep&cbmkKia=Kia&cbmkMazda=Mazda&cbmkMitsubishi=Mitsubishi&cbmkNissan=Nissan&cbmkRam=Ram&cbmkSubaru=Subaru&cbmkToyota=Toyota&cbmkVolkswagen=Volkswagen&cbmkVolvo=Volvo&cbmcfamilySedans=Family+Sedans&cbmclargeSedans=Large+Sedans&cbmcpickupTrucks=Pickup+Trucks&cbmcsportUtilityVehicles=Sport+Utility+Vehicles&cbmcminivans=Minivans&cbmcvans=Vans&minmsrpsel=0&maxmsrpsel=0&city=0&highway=0&combined=0&cbftdiesel=Diesel&YearSel=2018-2021&MakeSel=Acura%3B+BYD%3B+Chevrolet%3B+Chrysler%3B+Dodge%3B+Fiat%3B+Ford%3B+GMC%3B+Honda%3B+Hyundai%3B+Jeep%3B+Kia%3B+Mazda%3B+Mitsubishi%3B+Nissan%3B+Ram%3B+Subaru%3B+Toyota%3B+Volkswagen%3B+Volvo&MarClassSel=Family+Sedans%2C+Large+Sedans%2C+Pickup+Trucks%2C+Sport+Utility+Vehicles%2C+Minivans%2C+Vans&FuelTypeSel=Diesel&VehTypeSel=&TranySel=&DriveTypeSel=&CylindersSel=&MpgSel=000&sortBy=Comb&Units=&url=SearchServlet&opt=new&minmsrp=0&maxmsrp=0&minmpg=0&maxmpg=0&sCharge=&tCharge=&startstop=&cylDeact=&rowLimit=200')[0]
electric = pd.read_html('https://www.fueleconomy.gov/feg/PowerSearch.do?action=PowerSearch&year1=2018&year2=2021&cbmkAcura=Acura&cbmkBYD=BYD&cbmkChevrolet=Chevrolet&cbmkChrysler=Chrysler&cbmkDodge=Dodge&cbmkFiat=Fiat&cbmkFord=Ford&cbmkGMC=GMC&cbmkHonda=Honda&cbmkHyundai=Hyundai&cbmkJeep=Jeep&cbmkKia=Kia&cbmkMazda=Mazda&cbmkMitsubishi=Mitsubishi&cbmkNissan=Nissan&cbmkRam=Ram&cbmkSubaru=Subaru&cbmkToyota=Toyota&cbmkVolkswagen=Volkswagen&cbmkVolvo=Volvo&cbmcfamilySedans=Family+Sedans&cbmclargeSedans=Large+Sedans&cbmcpickupTrucks=Pickup+Trucks&cbmcsportUtilityVehicles=Sport+Utility+Vehicles&cbmcminivans=Minivans&cbmcvans=Vans&minmsrpsel=0&maxmsrpsel=0&city=0&highway=0&combined=0&cbvtelectric=Electric&YearSel=2018-2021&MakeSel=Acura%3B+BYD%3B+Chevrolet%3B+Chrysler%3B+Dodge%3B+Fiat%3B+Ford%3B+GMC%3B+Honda%3B+Hyundai%3B+Jeep%3B+Kia%3B+Mazda%3B+Mitsubishi%3B+Nissan%3B+Ram%3B+Subaru%3B+Toyota%3B+Volkswagen%3B+Volvo&MarClassSel=Family+Sedans%2C+Large+Sedans%2C+Pickup+Trucks%2C+Sport+Utility+Vehicles%2C+Minivans%2C+Vans&FuelTypeSel=&VehTypeSel=Electric&TranySel=&DriveTypeSel=&CylindersSel=&MpgSel=000&sortBy=Comb&Units=&url=SearchServlet&opt=new&minmsrp=0&maxmsrp=0&minmpg=0&maxmpg=0&sCharge=&tCharge=&startstop=&cylDeact=&rowLimit=200')[0]
e85 = pd.read_html('https://www.fueleconomy.gov/feg/PowerSearch.do?action=PowerSearch&year1=2018&year2=2021&cbmkAcura=Acura&cbmkBYD=BYD&cbmkChevrolet=Chevrolet&cbmkChrysler=Chrysler&cbmkDodge=Dodge&cbmkFiat=Fiat&cbmkFord=Ford&cbmkGMC=GMC&cbmkHonda=Honda&cbmkHyundai=Hyundai&cbmkJeep=Jeep&cbmkKia=Kia&cbmkMazda=Mazda&cbmkMitsubishi=Mitsubishi&cbmkNissan=Nissan&cbmkRam=Ram&cbmkSubaru=Subaru&cbmkToyota=Toyota&cbmkVolkswagen=Volkswagen&cbmkVolvo=Volvo&cbmcfamilySedans=Family+Sedans&cbmclargeSedans=Large+Sedans&cbmcpickupTrucks=Pickup+Trucks&cbmcsportUtilityVehicles=Sport+Utility+Vehicles&cbmcminivans=Minivans&cbmcvans=Vans&minmsrpsel=0&maxmsrpsel=0&city=0&highway=0&combined=0&cbfte85=E85&YearSel=2018-2021&MakeSel=Acura%3B+BYD%3B+Chevrolet%3B+Chrysler%3B+Dodge%3B+Fiat%3B+Ford%3B+GMC%3B+Honda%3B+Hyundai%3B+Jeep%3B+Kia%3B+Mazda%3B+Mitsubishi%3B+Nissan%3B+Ram%3B+Subaru%3B+Toyota%3B+Volkswagen%3B+Volvo&MarClassSel=Family+Sedans%2C+Large+Sedans%2C+Pickup+Trucks%2C+Sport+Utility+Vehicles%2C+Minivans%2C+Vans&FuelTypeSel=E85&VehTypeSel=&TranySel=&DriveTypeSel=&CylindersSel=&MpgSel=000&sortBy=Comb&Units=&url=SearchServlet&opt=new&minmsrp=0&maxmsrp=0&minmpg=0&maxmpg=0&sCharge=&tCharge=&startstop=&cylDeact=&rowLimit=200')[0]
gasoline = pd.read_html('https://www.fueleconomy.gov/feg/PowerSearch.do?action=PowerSearch&year1=2018&year2=2021&cbmkAcura=Acura&cbmkBYD=BYD&cbmkChevrolet=Chevrolet&cbmkChrysler=Chrysler&cbmkDodge=Dodge&cbmkFiat=Fiat&cbmkFord=Ford&cbmkGMC=GMC&cbmkHonda=Honda&cbmkHyundai=Hyundai&cbmkJeep=Jeep&cbmkKia=Kia&cbmkMazda=Mazda&cbmkMitsubishi=Mitsubishi&cbmkNissan=Nissan&cbmkRam=Ram&cbmkSubaru=Subaru&cbmkToyota=Toyota&cbmkVolkswagen=Volkswagen&cbmkVolvo=Volvo&cbmcfamilySedans=Family+Sedans&cbmclargeSedans=Large+Sedans&cbmcpickupTrucks=Pickup+Trucks&cbmcsportUtilityVehicles=Sport+Utility+Vehicles&cbmcminivans=Minivans&cbmcvans=Vans&minmsrpsel=0&maxmsrpsel=0&city=0&highway=0&combined=0&cbvtgasoline=Gasoline&YearSel=2018-2021&MakeSel=Acura%3B+BYD%3B+Chevrolet%3B+Chrysler%3B+Dodge%3B+Fiat%3B+Ford%3B+GMC%3B+Honda%3B+Hyundai%3B+Jeep%3B+Kia%3B+Mazda%3B+Mitsubishi%3B+Nissan%3B+Ram%3B+Subaru%3B+Toyota%3B+Volkswagen%3B+Volvo&MarClassSel=Family+Sedans%2C+Large+Sedans%2C+Pickup+Trucks%2C+Sport+Utility+Vehicles%2C+Minivans%2C+Vans&FuelTypeSel=&VehTypeSel=Gasoline&TranySel=&DriveTypeSel=&CylindersSel=&MpgSel=000&sortBy=Comb&Units=&url=SearchServlet&opt=new&minmsrp=0&maxmsrp=0&minmpg=0&maxmpg=0&sCharge=&tCharge=&startstop=&cylDeact=&rowLimit=1453')[0]

The web-scraped data has all kinds of formatting issues in pandas, so we will need to clean it up a bit. I wrote the following functions to do exactly this.

In [3]:
def convert_to_string (arr):
    """
    This function converts object arrays to string arrays
    
    Parameters:
    -----------
    arr : array of objects
    
    Returns:
    --------
    str_arr : float
            Array of strings
            
    """
    str_arr = np.array([])
    for a in arr:
        str_arr = np.append(str_arr, str(a))
    return str_arr


def split_dash (s):
    """
    This function splits a string along any dashes it might have.
    
    Parameters:
    -----------
    s : string
    
    Returns:
    --------
    s_arr : strings
            Array of strings
    """
    return re.split('[–]', s)


In [4]:
def get_avg_msrp (df):
    """
    This function takes the data table fron fueleconomy.gov
    and returns the average of the MSRP values listed there.
    
    Parameters:
    -----------
    df : pandas dataframe
        THe return value of pd.read_html()
    
    Returns:
    --------
    avg : float
        Average of the MSRP data
        
    """
    
    # Get rid of NaN MSRP values
    df = df[df['MSRP'].notna()]
    
    # Pick out MSRP entries that have a dollar sign
    df = df[df['MSRP'].str.contains('\$')]
    
    # Convert these to string form
    msrp_string = convert_to_string(df['MSRP'].values)
    
    # Many of the MSRPs are a given as a range between a lower and upper limit
    # of values. We want to take the average of this range.
    msrp_string = map(split_dash, msrp_string)
    msrp_num = np.array([])
    for s in msrp_string:
        if (len(s) == 2):
            s[0] = float(s[0][1:].replace(',',''))
            s[1] = float(s[1][1:].replace(',',''))
            s = (s[0] + s[1]) / 2
        else:
            s = float(s[0][1:].replace (',',''))
            
        msrp_num = np.append(msrp_num, s)
            
    return np.average(msrp_num)
    

With our functions defined and ready to go, we just need to call them with the relevant vehicle data:

In [9]:
get_avg_msrp(diesel)/15

2514.2083333333335

In [10]:
get_avg_msrp(electric)/15

2740.6666666666665

In [12]:
get_avg_msrp(e85)/15

2924.2732558139537

In [13]:
get_avg_msrp(gasoline)/15

1623.9814814814815